In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/PhD/GaussianProcessNode`


In [2]:
using Distributions, LinearAlgebra, FFTW, ToeplitzMatrices, Distances, KernelFunctions, BenchmarkTools
using ReactiveMP

In [3]:
#Sqexponential kernel with Δt argument
function stationary_vector(delta,v,l;jitter=1e-12)
    return v*exp.(-0.5.*delta.^2 / l) 
end

# function circularizes arrays
function circularize(arr)
    arrlength      = length(arr)
    circularlength = 2*(arrlength-1)
    
    circulararr    = Array{eltype(arr)}(undef,circularlength)
    
    @views circulararr[1:arrlength] .= view(arr,:,1)
    @inbounds [circulararr[i] = arr[arrlength - i%arrlength] for i=arrlength+1:circularlength]
    
    return circulararr
end

function Periodic_stationary_vector(delta,v,l,p; jitter=1e-12)
    return v * exp.(-2 * (sin.(π/p * abs.(delta))).^2 / l^2)
end


Periodic_stationary_vector (generic function with 1 method)

In [4]:
a = [1., 2., 3.]
circularized_a = circularize(a)
kernelmatrix(SqExponentialKernel(),circularized_a, circularized_a)

4×4 Matrix{Float64}:
 1.0       0.606531  0.135335  0.606531
 0.606531  1.0       0.606531  1.0
 0.135335  0.606531  1.0       0.606531
 0.606531  1.0       0.606531  1.0

In [6]:
N     = 4
delta = collect(0:1:N)
v     = 1.
l     = 1

circulariseddelta = circularize(delta)
uvect = stationary_vector(circulariseddelta,v,l) + Periodic_stationary_vector(circulariseddelta,v,l,1);
x     = zeros(length(circulariseddelta))
x[1:length(delta)] .= rand(length(delta));
# x = circularize(rand(length(delta)))
uvect;

In [7]:
uvect

8-element Vector{Float64}:
 2.0
 1.6065306597126334
 1.1353352832366128
 1.0111089965382423
 1.0003354626279024
 1.0111089965382423
 1.1353352832366128
 1.6065306597126334

In [114]:
circulantcov = Circulant(uvect);
# println("Circulant cov is posdef: " , isposdef(circulantcov))
# println("Circulant cov is symmetric: " , issymmetric(circulantcov))

200×200 Circulant{Float64}:
 1.0          0.606531     0.135335     …  0.135335     0.606531
 0.606531     1.0          0.606531        0.011109     0.135335
 0.135335     0.606531     1.0             0.000335463  0.011109
 0.011109     0.135335     0.606531        3.72665e-6   0.000335463
 0.000335463  0.011109     0.135335        1.523e-8     3.72665e-6
 3.72665e-6   0.000335463  0.011109     …  2.28973e-11  1.523e-8
 1.523e-8     3.72665e-6   0.000335463     1.26642e-14  2.28973e-11
 2.28973e-11  1.523e-8     3.72665e-6      2.57676e-18  1.26642e-14
 1.26642e-14  2.28973e-11  1.523e-8        1.92875e-22  2.57676e-18
 2.57676e-18  1.26642e-14  2.28973e-11     5.31109e-27  1.92875e-22
 ⋮                                      ⋱               
 2.57676e-18  1.92875e-22  5.31109e-27     2.28973e-11  1.26642e-14
 1.26642e-14  2.57676e-18  1.92875e-22     1.523e-8     2.28973e-11
 2.28973e-11  1.26642e-14  2.57676e-18     3.72665e-6   1.523e-8
 1.523e-8     2.28973e-11  1.26642e-14     0.00

In [99]:
truekernelmatrix = circulantcov[1:length(delta),1:length(delta)];
# submatrixA       = truekernelmatrix
# submatrixB       = circulantcov[1:length(delta),length(delta)+1:end]; #A and B are transpose of each other
# submatrixC       = circulantcov[length(delta)+1:end,1:length(delta)];
submatrixD       = circulantcov[length(delta)+1:end, length(delta)+1:end];


In [17]:
function returnFFTstructures(circularvector)
    N                   = length(circularvector)
    n                   = Int(N/2 + 1)
    Afftmatrix          = plan_rfft(circularvector)
    afft                = Vector{ComplexF64}(undef, n)
    bfft                = Vector{ComplexF64}(undef, n)
    cfft                = Vector{Float64}(undef, N)
    Ainvfftmatrix       = plan_irfft(afft, N)
    
    return Afftmatrix,Ainvfftmatrix,afft,bfft,cfft 
end


function fastmultiply(a, b , Afftmatrix, Ainvfftmatrix,afft,bfft,cfft)
   
    mul!(afft, Afftmatrix, a)
    mul!(bfft, Afftmatrix, b)
    
    afft .*= bfft
    return mul!(cfft, Ainvfftmatrix, afft)
end

function fastinversemultiply(a, b , Afftmatrix, Ainvfftmatrix,afft,bfft,cfft)
   
    mul!(afft, Afftmatrix, a)
    mul!(bfft, Afftmatrix, b)
    
    bfft ./= afft
    return mul!(cfft, Ainvfftmatrix, bfft)
end



function fastmahalanobis(a, b , Afftmatrix,afft,bfft)
    mul!(afft, Afftmatrix, a)
    mul!(bfft, Afftmatrix, b)
    return real((2*sum(afft .* abs.(bfft).^2) - (afft[1] * abs(bfft[1])^2 + afft[length(afft)] * abs(bfft[length(afft)])^2))/length(a))
end


function fastinvmahalanobis(a, b , Afftmatrix,afft,bfft)
    mul!(afft, Afftmatrix, a)
    mul!(bfft, Afftmatrix, b)
    
    return real((2*sum(abs.(bfft).^2 ./ afft) - (abs(bfft[1])^2/afft[1] + abs(bfft[length(afft)])^2/afft[length(afft)]))/length(a))
end

fastinvmahalanobis (generic function with 1 method)

In [18]:
Afftmatrix, Ainvfftmatrix,afft,bfft,cfft = returnFFTstructures(uvect);

In [116]:
test1 = rand(200);

In [117]:
@btime groundtruth = test1' * inv(circulantcov) * test1;

  54.167 μs (60 allocations: 20.05 KiB)


In [118]:
@btime fastcalculation = fastinvmahalanobis(uvect, test1, Afftmatrix,afft,bfft);

  1.450 μs (2 allocations: 1.78 KiB)


In [110]:
groundtruth  ≈ fastcalculation

true

In [47]:
xtrain = [.6, 1.1]
xtest  = [0.6, 1.1]
x      = vcat(xtrain,xtest)

Kxtrainxtrain  = stationary_vector([xtrain[i] - xtrain[j] for i=1:length(xtrain), j=1:length(xtrain)],v,l)
Kxtestxtest = stationary_vector([xtest[i] - xtest[j] for i=1:length(xtest), j=1:length(xtest)],v,l)
Kxx         = stationary_vector([xtest[i] - xtrain[j] for i=1:length(xtest), j=1:length(xtrain)],v,l)


deltatrain = xtrain .- xtrain[1]
deltatest  = xtest .- xtest[1]
delta      = vec([xtest[i] - xtrain[j] for i=1:length(xtest), j=1:length(xtrain)])

circulardeltatrain = circularize(deltatrain)
circulardeltatest  = circularize(deltatest);
circulardelta      = circularize(delta);


circulantKxtrainKxtrain = Circulant(stationary_vector(circulardeltatrain,v,l))
circulantKxtestKxtest   = Circulant(stationary_vector(circulardeltatest, v,l))
circulantKxx            = Circulant(stationary_vector(circulardelta,     v,l));

In [48]:
GOLD = Kxtestxtest - Kxx*inv(Kxtrainxtrain)*Kxx'

2×2 Matrix{Float64}:
 0.0  0.0
 0.0  0.0

In [49]:
delta

4-element Vector{Float64}:
  0.0
  0.5000000000000001
 -0.5000000000000001
  0.0

In [50]:
circulantKxx

6×6 Circulant{Float64}:
 1.0       0.882497  0.882497  1.0       0.882497  0.882497
 0.882497  1.0       0.882497  0.882497  1.0       0.882497
 0.882497  0.882497  1.0       0.882497  0.882497  1.0
 1.0       0.882497  0.882497  1.0       0.882497  0.882497
 0.882497  1.0       0.882497  0.882497  1.0       0.882497
 0.882497  0.882497  1.0       0.882497  0.882497  1.0

## See how I can apply this for gp 

In [9]:
xtrain = [1., 2., 3.]
xtest = [1., 2., 3.]

Ktrain = stationary_vector([xtrain[i] - xtrain[j] for i=1:length(xtrain), j=1:length(xtrain)],v,l)
Ktest = stationary_vector([xtest[i] - xtest[j] for i=1:length(xtest), j=1:length(xtest)],v,l)
Ktesttrain = stationary_vector([xtest[i] - xtrain[j] for i=1:length(xtest), j=1:length(xtrain)],v,l)

3×3 Matrix{Float64}:
 1.0       0.606531  0.135335
 0.606531  1.0       0.606531
 0.135335  0.606531  1.0

In [23]:
delta_xtrain = xtrain .- xtrain[1]
circular_delta_xtrain = circularize(delta_xtrain)
stationary_vectrain = stationary_vector(circular_delta_xtrain,v,l)
circulantmatrix_Ktrain = Circulant(stationary_vectrain)

4×4 Circulant{Float64}:
 1.0       0.606531  0.135335  0.606531
 0.606531  1.0       0.606531  0.135335
 0.135335  0.606531  1.0       0.606531
 0.606531  0.135335  0.606531  1.0

In [45]:
#create noisy circulantmatrix 
Σ_noise = Diagonal(circularize(0.001*rand(length(delta_xtrain))))
noisy_circulantmatrix_Ktrain = circulantmatrix_Ktrain + Σ_noise

4×4 Matrix{Float64}:
 1.00069   0.606531  0.135335  0.606531
 0.606531  1.00047   0.606531  0.135335
 0.135335  0.606531  1.00067   0.606531
 0.606531  0.135335  0.606531  1.00047

In [33]:
test = rand(length(circular_delta_xtrain))

4-element Vector{Float64}:
 0.4805587179773715
 0.31234857967520624
 0.5188391023586912
 0.055989195485337206

In [34]:
Afftmatrix, Ainvfftmatrix,afft,bfft,cfft = returnFFTstructures(circular_delta_xtrain);

In [46]:
vec_gt = inv(noisy_circulantmatrix_Ktrain) * test 

4-element Vector{Float64}:
 -1.9211415613763048
  2.3385210750349894
 -1.876950122500393
  2.042198857893046

In [47]:
vec_fast = fastinversemultiply(stationary_vectrain,test,Afftmatrix,Ainvfftmatrix,afft,bfft,cfft)

4-element Vector{Float64}:
 -1.9062904209295433
  2.3236027889188797
 -1.8620184810213083
  2.027118638218856

In [48]:
norm(vec_gt - vec_fast)

0.029891111988236393